# 제약주의 임상시험 직후 주가 반응 이벤트 스터디

식약처 의약품 임상시험 리스트(link: https://nedrug.mfds.go.kr/searchClinic) 에서 상장기업을 골라낸 뒤 해당 임상시험 허가/불허 날짜를 찾고 분석에 적절한 종목을 선정한다. 
- 임상시험 허가심사 및 임상시험 결과가 미리 예측 가능하지 않아야 하며
- 임상시험 이외의 외부요인에 주가가 영향을 많이 받지 않는 종목이어야 한다. (가령, 의약품 말고도 화장품/마스크 등 다른 사업을 많이 가지고 있는 경우.) 

해당 날짜의 임상 허가/불허 뉴스가 최초 게시된 시각을 알아낸다. (분 단위)

그 시각 전후의 주가를 잘 관찰하여 적절한 estimation window, event window를 설정한다. 

한 종목에서 임상 뉴스에 대한 주가의 반응이 얼마나 빨리, 얼마나 나타나는지, 종목간의 차이는 어떤지 알아본다. 

## 프로젝트의 해결해야 할 이슈

- 현업에서 일반적으로 바이오/제약 관련 이벤트 스터디가 어떻게 이뤄지고 있는지 확인해야 한다. 
- 아카데믹에서 일반적으로 바이오/제약 관련 이벤트 스터디가 어떻게 이뤄지고 있는지 확인해야 한다. 
- 임상시험 허가/불허가 아닌 임상시험 성공/실패 데이터를 구하는 것이 더 영향력이 크고, 중요할 것으로 생각된다. 
    - 관련하여, 금융위는 앞으로는 투자자의 혼란을 막기 위해 '임상시험 성공'이라는 표현을 사용할 수 없게했다. 그 말은 지금까지의 임상시험 성공/실패 관련 기사나 공시에는 객관적으로 성공/실패를 알기 힘든, 오해를 불러일으킬 수 있는 표현이 사용되었다고 해석할 수 있다. (기사 참고: http://biospectator.com/view/news_view.php?varAtcId=9502) 
- 이벤트 스터디와 관련하여, 어떻게 객관적으로 estimation window, event window를 설정하고 임상시험의 효과를 검증할 수 있을지 고민해봐야 한다. 
- 1분단위 고빈도 주가 데이터라는 점을 고려하여 정상수익률 모형을 어떻게 설정해야 하는지 고민해봐야 한다. 

In [1]:
import win32com.client

import pandas as pd

In [2]:
instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect) # 1이면 성공

1


In [ ]:
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")